## Random Forest Model for WiDS Datathon Competition
Score: 93.197

In [41]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
import pandas as pd
import numpy as np


# read in data
train_data = pd.read_csv("/kaggle/input/WiDSWorldWide_GlobalDathon26/train.csv")
test_data = pd.read_csv("/kaggle/input/WiDSWorldWide_GlobalDathon26/test.csv")
train_data.head()

,event_id,num_perimeters_0_5h,dt_first_last_0_5h,low_temporal_resolution_0_5h,area_first_ha,area_growth_abs_0_5h,area_growth_rel_0_5h,area_growth_rate_ha_per_h,log1p_area_first,log1p_growth,...,dist_fit_r2_0_5h,alignment_cos,alignment_abs,cross_track_component,along_track_speed,event_start_hour,event_start_dayofweek,event_start_month,time_to_hit_hours,event
0,10892457,3,4.265188,0,79.696304,2.875935,0.036086,0.674281,4.390693,1.354787,...,0.886373,-0.054649,0.054649,-1.937219,-0.106026,19,4,5,18.892512,0
1,11757157,2,1.169918,0,8.946749,0.000000,0.000000,0.000000,2.297246,0.000000,...,0.000000,-0.568898,0.568898,-0.000000,-0.000000,4,4,6,22.048108,1
2,11945086,4,4.777526,0,106.482638,0.000000,0.000000,0.000000,4.677329,0.000000,...,0.000000,0.882385,0.882385,0.000000,0.000000,22,4,8,0.888895,1
3,12044083,1,0.000000,1,67.631125,0.000000,0.000000,0.000000,4.228746,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,20,5,8,60.953021,0
4,12052347,2,4.975273,0,35.632874,0.000000,0.000000,0.000000,3.600946,0.000000,...,0.000000,0.934634,0.934634,-0.000000,0.000000,21,5,7,44.990274,0


In [42]:
print(train_data.describe(include='all')) #view all column stats

           event_id  num_perimeters_0_5h  dt_first_last_0_5h  \
count  2.210000e+02           221.000000          221.000000   
mean   5.384397e+07             2.063348            0.979869   
std    2.507456e+07             2.578859            1.738052   
min    1.089246e+07             1.000000            0.000000   
25%    3.209326e+07             1.000000            0.000000   
50%    5.244094e+07             1.000000            0.000000   
75%    7.457274e+07             2.000000            1.356107   
max    9.933973e+07            17.000000            4.994457   

       low_temporal_resolution_0_5h  area_first_ha  area_growth_abs_0_5h  \
count                    221.000000     221.000000            221.000000   
mean                       0.728507     619.131641             26.332398   
std                        0.445739    1447.723668            187.437018   
min                        0.000000       0.037525             -0.000022   
25%                        0.000000      25

In [43]:
# view all of the columns names in the dataset
train_data.columns

Index(['event_id', 'num_perimeters_0_5h', 'dt_first_last_0_5h',
       'low_temporal_resolution_0_5h', 'area_first_ha', 'area_growth_abs_0_5h',
       'area_growth_rel_0_5h', 'area_growth_rate_ha_per_h', 'log1p_area_first',
       'log1p_growth', 'log_area_ratio_0_5h', 'relative_growth_0_5h',
       'radial_growth_m', 'radial_growth_rate_m_per_h',
       'centroid_displacement_m', 'centroid_speed_m_per_h',
       'spread_bearing_deg', 'spread_bearing_sin', 'spread_bearing_cos',
       'dist_min_ci_0_5h', 'dist_std_ci_0_5h', 'dist_change_ci_0_5h',
       'dist_slope_ci_0_5h', 'closing_speed_m_per_h',
       'closing_speed_abs_m_per_h', 'projected_advance_m',
       'dist_accel_m_per_h2', 'dist_fit_r2_0_5h', 'alignment_cos',
       'alignment_abs', 'cross_track_component', 'along_track_speed',
       'event_start_hour', 'event_start_dayofweek', 'event_start_month',
       'time_to_hit_hours', 'event'],
      dtype='object')

In [44]:
#find correlations between all attributes and our label time_to_hit_hours; closer to 1 or -1 = strong correlation, close to 0 = not much predictive value
correlations = train_data.corr()['time_to_hit_hours'].sort_values(ascending=False)
print(correlations) 

time_to_hit_hours               1.000000
low_temporal_resolution_0_5h    0.442236
spread_bearing_cos              0.370451
dist_min_ci_0_5h                0.324876
area_first_ha                   0.211212
log1p_area_first                0.192067
dist_accel_m_per_h2             0.138790
dist_change_ci_0_5h             0.131773
dist_slope_ci_0_5h              0.125267
event_start_dayofweek           0.123600
event_start_month               0.057934
cross_track_component           0.052684
event_id                        0.003129
along_track_speed              -0.023527
alignment_cos                  -0.079847
event_start_hour               -0.080910
closing_speed_m_per_h          -0.130362
projected_advance_m            -0.131773
area_growth_rel_0_5h           -0.157423
relative_growth_0_5h           -0.157423
dist_std_ci_0_5h               -0.160923
area_growth_abs_0_5h           -0.164590
closing_speed_abs_m_per_h      -0.166876
area_growth_rate_ha_per_h      -0.177191
centroid_speed_m

In [45]:
train_data.dtypes #check if there are any strings in the data set --> need to one hot encode if so

event_id                          int64
num_perimeters_0_5h               int64
dt_first_last_0_5h              float64
low_temporal_resolution_0_5h      int64
area_first_ha                   float64
area_growth_abs_0_5h            float64
area_growth_rel_0_5h            float64
area_growth_rate_ha_per_h       float64
log1p_area_first                float64
log1p_growth                    float64
log_area_ratio_0_5h             float64
relative_growth_0_5h            float64
radial_growth_m                 float64
radial_growth_rate_m_per_h      float64
centroid_displacement_m         float64
centroid_speed_m_per_h          float64
spread_bearing_deg              float64
spread_bearing_sin              float64
spread_bearing_cos              float64
dist_min_ci_0_5h                float64
dist_std_ci_0_5h                float64
dist_change_ci_0_5h             float64
dist_slope_ci_0_5h              float64
closing_speed_m_per_h           float64
closing_speed_abs_m_per_h       float64


In [46]:
#check for missing data for each of the columns
missing_data = train_data.isnull().sum()
print(missing_data[missing_data > 0])

Series([], dtype: int64)


In [48]:
#create new features that have more predictive power for predicting the label based on existing features
#reference: https://www.kaggle.com/code/loopassembly/0-97092-450-model-fold-fused-survival-engine
def create_features(df):
    result = df.copy()
    dist = result['dist_min_ci_0_5h'].clip(lower=1)
    speed = result['closing_speed_m_per_h']
    perimeters = result['num_perimeters_0_5h']
    area_first = result['area_first_ha']

    #fire risk doesn't decrease linearly with distance  --> log and sqrt distance compress large differences, helps with treating far / very far fires more similarly
    result['log_distance'] = np.log1p(dist)
    result['sqrt_distance'] = np.sqrt(dist)

    #flag immediate threats through 1/d --> close fires will have very high values, far fires will have very low values
    result['inv_distance'] = 1 / (dist / 1000 + 0.1)
    result['inv_distance_sq'] = result['inv_distance'] ** 2
    
    result['dist_km'] = dist / 1000
    result['dist_km_sq'] = (dist / 1000) ** 2
    result['dist_rank'] = dist.rank(pct=True)
    fire_radius = np.sqrt(area_first * 10000 / np.pi)
    result['radius_to_dist'] = fire_radius / dist
    result['area_to_dist_ratio'] = area_first / (dist / 1000 + 0.1)
    result['log_area_dist_ratio'] = np.log1p(area_first) - np.log1p(dist)
    result['has_movement'] = (perimeters > 1).astype(float)
    closing_pos = speed.clip(lower=0)
    
    
    radial_growth = result['radial_growth_rate_m_per_h'].clip(lower=0)
    effective_closing = closing_pos + radial_growth
    result['effective_closing_speed'] = effective_closing
    result['threat_score'] = result['alignment_abs'] * speed / np.log1p(dist)
    result['fire_urgency'] = perimeters * speed
    result['growth_intensity'] = result['area_growth_rate_ha_per_h'] * perimeters

    #useful for calculating time before arrival
    #eta effective --> distance / (speed + growth), eta hours --> distance / speed
    #use 9999 for fires where speed is <=0 --> 9999=fire is not really coming at all
    result['eta_hours'] = np.where(closing_pos > 0.01, dist / closing_pos, 9999).clip(max=9999)
    result['eta_effective'] = np.where(effective_closing > 0.01, dist / effective_closing, 9999).clip(max=9999)

    #bin zones so then the model doesn't have to use as much data to learn that less than 5km => critical / need to evacuate
    result['zone_critical'] = (dist < 5000).astype(float)
    result['zone_warning'] = ((dist >= 5000) & (dist < 10000)).astype(float)
    result['zone_safe'] = (dist >= 10000).astype(float)

    #capture diurnal cycle, as fires in the afternoon / summer can spread more compared to fires at other time periods
    result['is_summer'] = result['event_start_month'].isin([6, 7, 8]).astype(float)
    result['is_afternoon'] = ((result['event_start_hour'] >= 12) & (result['event_start_hour'] < 20)).astype(float)

    result['log_eta'] = np.log1p(result['eta_hours'].clip(0, 9999))
    drop_cols = ['relative_growth_0_5h', 'projected_advance_m', 'centroid_displacement_m',
                 'centroid_speed_m_per_h', 'closing_speed_abs_m_per_h', 'area_growth_abs_0_5h']
    result = result.drop(columns=[c for c in drop_cols if c in result.columns])
    result = result.replace([np.inf, -np.inf], np.nan).fillna(0)
    return result

train_data = create_features(train_data)
test_data = create_features(test_data)

In [49]:
# create columns for 12, 24, 28, and 72 hours based on time_to_hit_hours, since we have to predict whether a wildfire will threaten
# an evacuation zone for these four time periods
train_data["prob12hr"] = (train_data["time_to_hit_hours"] <= 12).astype(int)
train_data["prob24hr"] = (train_data["time_to_hit_hours"] <= 24).astype(int)
train_data["prob48hr"] = (train_data["time_to_hit_hours"] <= 48).astype(int)
train_data["prob72hr"] = (train_data["time_to_hit_hours"] <= 72).astype(int)

In [50]:
# partition features and labels
y = train_data[["prob12hr", "prob24hr", "prob48hr", "prob72hr"]]
X = train_data.drop(["prob12hr", "prob24hr", "prob48hr", "prob72hr", "time_to_hit_hours", "event"], axis=1)

In [51]:
#find best parameters for a random forest model

best_params = {}

rf = RandomForestClassifier(random_state=42)

param_grid = {
    'n_estimators': [100, 300, 500],
    'max_depth': [5, 10, 15, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 5, 10],
    'max_features': ['sqrt', 'log2'] 
}

random_search = RandomizedSearchCV(
    estimator=rf, 
    param_distributions=param_grid, 
    n_iter=20,           
    cv=5, 
    n_jobs=-1, 
    scoring='roc_auc', 
    random_state=42
)

random_search.fit(X, train_data["prob12hr"])
best_params['prob12hr'] = random_search.best_params_
print(f"Best parameters for 12h: {random_search.best_params_}")

random_search.fit(X, train_data["prob24hr"])
best_params['prob24hr'] = random_search.best_params_
print(f"Best parameters for 24h: {random_search.best_params_}")

random_search.fit(X, train_data["prob48hr"])
best_params['prob48hr'] = random_search.best_params_
print(f"Best parameters for 48h: {random_search.best_params_}")

# throws an error since prob72hr has little variation
# random_search.fit(X, train_data["prob72hr"])
# print(f"Best parameters for 72h: {random_search.best_params_}")


Best parameters for 12h: {'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 10, 'max_features': 'sqrt', 'max_depth': None}
Best parameters for 24h: {'n_estimators': 100, 'min_samples_split': 10, 'min_samples_leaf': 5, 'max_features': 'sqrt', 'max_depth': 5}
Best parameters for 48h: {'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': 15}


In [54]:
# train + create predictions based on the test dataset
labels = ["prob12hr", "prob24hr", "prob48hr", "prob72hr"]
models = {} # dictionary, key = time period (ex. prob12h), label = model trained using one of the four labels (ex. prob12h)

for time_period in labels:
    if time_period != 'prob72hr':
        model = RandomForestClassifier(
            n_estimators=best_params[time_period]['n_estimators'],
            max_depth=5,
            min_samples_split=best_params[time_period]['min_samples_split'], 
            min_samples_leaf=best_params[time_period]['min_samples_leaf'],
            max_features=best_params[time_period]['max_features'],
            random_state=42
        )
    else:
        model = RandomForestClassifier(
            n_estimators=200,
            max_depth=5,
            min_samples_split=3, 
            min_samples_leaf=1,
            max_features='log2',
            random_state=42
        )
    model.fit(X, train_data[time_period])
    models[time_period] = model

predictions = {}
X_test = test_data[X.columns]

for time_period in labels:
    prob = models[time_period].predict_proba(X_test) #returns two columns -> prob[column = 0] = fire will not reach, prob[column = 1] fire will reach
    
    if prob.shape[1] == 2:
        predictions[time_period] = prob[:, 1]
    else: #handle edge case where only one column exists (for 72h, fire always reaches, therefore prob only has one columns -> was not trained with any data where fire did not reach in 72 hr)
         predictions[time_period] = prob[:, 0]

In [55]:
output = pd.DataFrame({
    "event_id": test_data["event_id"],
    "prob_12h": predictions["prob12hr"],
    "prob_24h": predictions["prob24hr"],
    "prob_48h": predictions["prob48hr"],
    "prob_72h": predictions["prob72hr"]
})

# save to a csv file, don't include the index column (would add an extra 6th column to submission.csv)
output.to_csv("submission.csv", index=False)

print(output.head())

   event_id  prob_12h  prob_24h  prob_48h  prob_72h
0  10662602  0.023549  0.267282  0.379779       1.0
1  13353600  0.602468  0.946287  0.968808       1.0
2  13942327  0.020305  0.140133  0.564565       1.0
3  16112781  0.593392  0.933168  0.976546       1.0
4  17132808  0.204219  0.254973  0.716812       1.0
